In [231]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet

In [232]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [233]:
from nltk import pos_tag

In [234]:
df = pd.read_csv('Datasets/Test.csv')
X_test = df['text']

In [235]:
df = pd.read_csv('Datasets/Train.csv')
X_train = df['text']
Y_train = df['airline_sentiment'].replace({
    'negative' : 0,
    'neutral' : 1,
    'positive': 2
})

In [236]:
def simplify(pos):
    if pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('R'):
        return wordnet.ADV
    elif pos.startswith('A'):
        return wordnet.ADJECTIVE
    elif pos.startswith('V'):
        return wordnet.VERB
    else:
        return wordnet.NOUN

In [237]:
def change_words(document):
    stop = stopwords.words('english') + list(string.punctuation)
    orginal_words = word_tokenize(document)
    words = []
    for word in orginal_words:
        if word.lower() not in stop:
            pos = pos_tag([word])[0][1]
            word = lemmatizer.lemmatize(word,pos= simplify(pos))
            words.append(word.lower())
    return words

In [238]:
X_train = [change_words(doc) for doc in X_train ]
X_test = [change_words(doc) for doc in X_test ]

In [239]:
X_train[0]

['southwestair',
 'schedule',
 'morning',
 '2',
 'day',
 'fact',
 'yes',
 '..',
 'sure',
 'even',
 'flight',
 'one',
 'cancelled',
 'flightled']

In [240]:
def change_into_text(words):
    return ' '.join(words)

In [241]:
training_data_X = [ change_into_text(words) for words in X_train]
testing_data_X = [ change_into_text(words) for words in X_test]

In [242]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features=3000,ngram_range=(1,2))
count_vect.fit(training_data_X)

CountVectorizer(max_features=3000, ngram_range=(1, 2))

In [243]:
X_train= count_vect.transform(training_data_X)
X_train = X_train.todense()
X_test = count_vect.transform(testing_data_X)
X_train

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [244]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train,Y_train)

LogisticRegression(max_iter=2000)

In [245]:
ans= classifier.predict(X_test.todense())

In [246]:
ans = pd.Series(ans).replace({
    0 : 'negative', 
    1 : 'neutral',
    2 : 'positive'
})

In [247]:
ans.to_csv('Datasets/Predition.csv',index=False,header=False)